In [3]:
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools

from __future__ import print_function

from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_samples, silhouette_score

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

In [4]:
import os, sys
sys.path.append(os.path.split(os.getcwd())[0])

from WordVector import WordVector

In [5]:
import data_helper
_,_,relation = data_helper.get_triplet()
relation[:5]

['cause', 'cause', 'cause', 'cause', 'cause']

In [6]:
relation = list(set(relation))
relation[:5]

['result', 'prevent', 'accompany', 'gram-negative', 'initiate']

In [7]:
wv = WordVector(relation)

Loading a pre-trained model...
Load success!


In [8]:
wv.word[:5]

['result', 'prevent', 'accompany', 'gram-negative', 'initiate']

In [9]:
wv.vector[0]

array([-0.74980611,  3.26506686,  2.16919637, -1.6358254 , -2.86456966,
       -0.72669113, -1.68594217,  1.36314213, -1.81251729, -0.08911414,
        2.74543476,  0.65284687,  0.85167217, -0.81214362, -0.86352259,
        1.5679059 ,  2.04781485,  1.12430871, -0.79476273,  2.59588313,
        1.61632693, -4.25633717, -1.68846977, -1.08155656, -0.93635136,
       -1.57342398, -5.58548832,  2.67990899,  0.31876048, -0.29126978,
        1.64317811,  1.78237104,  0.8678447 , -0.9605661 ,  2.86079121,
       -2.44644594,  1.60718644, -0.1497878 , -3.70028758,  2.22714853,
       -1.93805015, -2.48986506,  1.86366177, -3.2291317 ,  1.66404438,
        1.29343855,  0.79689205, -1.55539024,  2.04541111,  0.22868253,
       -2.60663724, -1.94535196,  0.54713982,  1.02404118,  1.67203939,
       -2.21519685,  3.67146468,  2.04408407, -3.97368431, -1.58429825,
        2.17803407, -1.11728048, -1.87352455, -3.55320001, -0.86778229,
       -2.55176663,  3.50662804,  0.15311995,  0.38133213, -3.41

# PCA: 100dim -> 3dim

In [10]:
from DimensionReduction import DimensionReduction
dr = DimensionReduction()
vec_3d = dr.PCA(wv.vector, dim=3)
vec_3d[:3]

Explained variation per principal component: [ 0.11155299  0.07719805  0.06918809]
Average of Explained variations: 0.2579391300678253


array([[ -0.16393648,   1.28157234,   0.16697837],
       [-10.41118908,  -4.94548893,   3.22475696],
       [ -8.69082069,  -8.82754135,   0.07148482]], dtype=float32)

# Silhouette Coefficient Analysis

In [12]:
range_n_clusters = [2, 3, 4, 5, 6, 7, 8, 9, 10]
colors=["rgb(255,0,0)", "rgb(0,255,0)", "rgb(0,0,255)", "rgb(255,255,0)", "rgb(255,0,255)", 
        "rgb(0,255,255)", "rgb(127,0,255)", "rgb(255,128,0)","rgb(255,0,127)", "rgb(128,128,128)"]

In [13]:
figures = []

for n_clusters in range_n_clusters:
    # Create a subplot with 1 row and 2 columns
    fig = tools.make_subplots(rows=1, cols=2,
                              print_grid=False,
                              specs=[[{'is_3d': False}, {'is_3d': True}]])

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    fig['layout']['xaxis1'].update(title='The silhouette coefficient values', range=[-0.1, 1])
   
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    fig['layout']['yaxis1'].update(title='Cluster label',
                                   showticklabels=False,
                                   range=[0, len(vec_3d) + (n_clusters + 1) * 10])

    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    clusterer = KMeans(n_clusters=n_clusters)
    cluster_labels = clusterer.fit_predict(vec_3d)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(vec_3d, cluster_labels)
    print("For n_clusters =", n_clusters,
          "The average silhouette_score is :", silhouette_avg)

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(vec_3d, cluster_labels)
    y_lower = 10
    
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = \
            sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        # colors = cm.spectral(cluster_labels.astype(float) / n_clusters)

        filled_area = go.Scatter(y=np.arange(y_lower, y_upper),
                                 x=ith_cluster_silhouette_values,
                                 mode='lines',
                                 showlegend=False,
                                 line=dict(width=0.5,
                                          color=colors[i]),
                                 fill='tozerox')
        fig.append_trace(filled_area, 1, 1)
        
        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples
        

    # The vertical line for average silhouette score of all the values
    axis_line = go.Scatter(x=[silhouette_avg],
                           y=[0, len(vec_3d) + (n_clusters + 1) * 10],
                           showlegend=False,
                           mode='lines',
                           line=dict(color="red", dash='dash',
                                     width =1) )

    fig.append_trace(axis_line, 1, 1)
    
    
    
    
    
    
    # 2nd Plot showing the actual clusters formed
    
    labeled_vec = list(zip(vec_3d, cluster_labels))
    # Labeling the clusters
    centers_ = clusterer.cluster_centers_
    for i in range(n_clusters):
        ith_cluster_vector = np.array([v for (v,c) in labeled_vec if c == i])
        # colors = matplotlib.colors.colorConverter.to_rgb(cm.spectral(float(i) / n_clusters))
        # colors = 'rgb'+str(colors)
        
        clusters = go.Scatter3d(x=ith_cluster_vector[:, 0], 
                                y=ith_cluster_vector[:, 1], 
                                z=ith_cluster_vector[:, 2],
                                showlegend=False,
                                mode='markers',
                                marker=dict(
                                    color=colors[i],
                                    size=3,
                                    line=dict(
                                        width=0.5
                                    )
                                )
                               )
        fig.append_trace(clusters, 1, 2)
        
        # Draw white circles at cluster centers
        centers = go.Scatter3d(x=centers_[i][0], 
                               y=centers_[i][1],
                               z=centers_[i][2],
                               showlegend=False,
                               mode='markers',
                               marker=dict(
                                   color=colors[i], size=5,
                                   line=dict(color='black', width=1))
                               )

        fig.append_trace(centers, 1, 2)
    
                      
        
        
        

    fig['layout'].update(title="Silhouette analysis for KMeans clustering on sample data "
                         "with n_clusters = %d" % n_clusters)
    
    figures.append(fig)

For n_clusters = 2 The average silhouette_score is : 0.291592
For n_clusters = 3 The average silhouette_score is : 0.325648
For n_clusters = 4 The average silhouette_score is : 0.378724
For n_clusters = 5 The average silhouette_score is : 0.326929
For n_clusters = 6 The average silhouette_score is : 0.325085
For n_clusters = 7 The average silhouette_score is : 0.315989
For n_clusters = 8 The average silhouette_score is : 0.331336
For n_clusters = 9 The average silhouette_score is : 0.324034
For n_clusters = 10 The average silhouette_score is : 0.327786


In [14]:
py.iplot(figures[0])

In [15]:
py.iplot(figures[1])

In [16]:
py.iplot(figures[2])